In [ ]:
import pandas as pd
import numpy as np
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns
import wrangle_COSA
pd.set_option('display.max_columns', None)

In [ ]:
wrangle.wrangle_saws()

In [ ]:
air_df = wrangle.clean_air()
air_df.datetime = pd.to_datetime(air_df.datetime)
air_df = air_df.set_index('datetime')
air_df = air_df.sort_index()
air1_df = air_df[(air_df['Pm1_0'] < 500) & (air_df['Pm1_0'] > -50)]

In [ ]:
plt.plot(air_df.index, air_df.SO2)

In [ ]:
plt.plot(air_df.index, air_df.O3)

In [ ]:
plt.plot(air_df.index, air_df.CO)

In [ ]:
plt.plot(air_df.index, air_df.NO2)

In [ ]:
weather_df = wrangle.wrangle_weather()

In [ ]:
weather_df.dewpoint_celsius.isna().sum()

In [ ]:
weather_df['wind'] = weather_df['wind'].str.extract('(\d+)', expand=False)
weather_df['visibility'] = weather_df['visibility'].str.extract('(\d+)', expand=False)
weather_df['wind'] = weather_df['wind'].fillna(0)
weather_df['wind'] = weather_df['wind'].apply(lambda x: int(x))
weather_df['visibility'] = weather_df['visibility'].apply(lambda x: int(x))

In [ ]:
weather_df.describe()

In [ ]:
air_df.describe()

In [ ]:
def full_daily_COSA_dataframe():
    sound_df = wrangle.wrangle_sound()
    sound_df = sound_df.set_index('DateTime')
    sound_df = sound_df.sort_index()
    flood_df = wrangle.clean_flood()
    flood_df = flood_df.set_index('datetime')
    weather_df = wrangle.wrangle_weather()
    air_df = wrangle.clean_air()
    air_df.datetime = pd.to_datetime(air_df.datetime)
    air_df = air_df.set_index('datetime')
    air_df = air_df.sort_index()
    weather_day_df = weather_df.resample('D', on='datetime').mean()
    flood_day_df = flood_df.resample('D').mean()
    sound_day_df = sound_df.resample('D').mean()
    air_day_df = air_df.resample('D').mean().drop(columns = ['hour', 'weekday', 'CO_24hr', 'Pm_25_24hr', 'Pm_10_24hr', 'SO2', 'O3', 'NO2'])
    air2_5 = air_df.drop(air_df.columns.difference(['Pm2_5', 'AQI_pm2_5']), 1)
    air10 = air_df.drop(air_df.columns.difference(['Pm10', 'AQI_pm10']), 1)
    airCO = air_df.drop(air_df.columns.difference(['CO', 'AQI_CO']), 1)
    series2_5 = air2_5.resample('D').max().rename(columns = {'AQI_pm2_5': 'most_hazardous_pm2.5_level'})['most_hazardous_pm2.5_level']
    series10 = air10.resample('D').max().rename(columns = {'AQI_pm10': 'most_hazardous_pm10_level'})['most_hazardous_pm10_level']
    seriesCO = airCO.resample('D').max().rename(columns = {'AQI_CO': 'most_hazardous_CO_level'})['most_hazardous_CO_level']
    hazards = pd.DataFrame(series2_5).join(series10).join(seriesCO)
    df = weather_day_df.join(air_day_df).join(hazards).join(sound_day_df).join(flood_day_df)
    df = df.round({'celsius': 2, 'farenheit': 2, 'humidity': 2, 'dewpoint_celsius': 2, 'dewpoint_farenheit': 2,
          'pressure': 2, 'NoiseLevel_db': 2, 'sensor_to_water_feet': 2, 'sensor_to_water_meters': 2,
          'sensor_to_ground_feet': 2, 'sensor_to_ground_meters': 2, 'flood_depth_feet': 2,
          'flood_depth_meters': 2})
    return df

In [ ]:
df = wrangle.full_daily_COSA_dataframe()

In [ ]:
df

In [ ]:
ax = df.Pm1_0.groupby(df.index.day_name()).mean().plot.bar(width=.9, ec='black')
plt.xticks(rotation=0)
ax.set(title='Average PM 1.0 Level by Day of Week', xlabel='Day of Week', ylabel='PM 1.0 Level')

In [ ]:
ax = df.Pm2_5.groupby(df.index.day_name()).mean().plot.bar(width=.9, ec='black')
plt.xticks(rotation=0)
ax.set(title='Average PM 2.5 Level by Day of Week', xlabel='Day of Week', ylabel='PM 2.5 Level')

In [ ]:
ax = df.Pm10.groupby(df.index.day_name()).mean().plot.bar(width=.9, ec='black')
plt.xticks(rotation=0)
ax.set(title='Average PM 10 Level by Day of Week', xlabel='Day of Week', ylabel='PM 10 Level')

In [ ]:
ax = df.CO.groupby(df.index.day_name()).mean().plot.bar(width=.9, ec='black')
plt.xticks(rotation=0)
ax.set(title='Average PM Carbon Monoxide Level by Day of Week', xlabel='Day of Week', ylabel='CO Level')

### SAWS

In [ ]:
saws_df = wrangle.wrangle_saws()
saws_df['year_month'] = '20' + saws_df['year_month']
saws_df['year_month'] = pd.to_datetime(saws_df['year_month'])
saws_month_year = saws_df.set_index('year_month').resample('M').mean().drop(columns = ['zipcode'])
saws_df

In [ ]:
saws_month_year_sum = saws_df.set_index('year_month').resample('M').sum().drop(columns = ['zipcode'])
saws_month_year_sum

In [ ]:
def show_saws():
    saws_df = wrangle.wrangle_saws()
    saws_df['year_month'] = '20' + saws_df['year_month']
    saws_df['year_month'] = pd.to_datetime(saws_df['year_month'])
    saws_month_year = saws_df.set_index('year_month').resample('M').mean().drop(columns = ['zipcode'])
    saws_month_year['Date'] = pd.to_datetime(saws_month_year.index)
    saws_month_year['Mon_Year'] = saws_month_year['Date'].dt.strftime('%b-%Y')
    saws_month_year_sum = saws_df.set_index('year_month').resample('M').sum().drop(columns = ['zipcode'])
    saws_month_year_sum['Date'] = pd.to_datetime(saws_month_year_sum.index)
    saws_month_year_sum['Mon_Year'] = saws_month_year_sum['Date'].dt.strftime('%b-%Y')
    saws_places = saws_df.groupby('location').mean()
    saws_places = saws_places.drop(columns =['zipcode'])
    saws_places_sum = saws_df.groupby('location').sum()
    import calendar
    saws_year_month_mean = saws_df.gallons_consumed.groupby(saws_df['year_month'].dt.month).mean()
    saws_year_month_mean = pd.DataFrame(saws_year_month_mean)
    saws_year_month_mean['month'] = saws_year_month_mean.index
    saws_year_month_mean['month'] = saws_year_month_mean['month'].apply(lambda x: calendar.month_abbr[x])
    
    plt.subplots(5, 1, figsize=(24, 40), sharey=True)
    plt.subplots_adjust(hspace=.6)
    sns.set(style="darkgrid")
        
    plt.subplot(5, 1, 1)
    plt.title('Average Monthly Water Use by Street')
    plt.xticks(rotation = 90)
    sns.barplot(data = saws_places, x = saws_places.index, y = 'gallons_consumed', palette = "viridis")
    plt.xlabel('Street Name')
    plt.ylabel('Average Monthly Gallons')
    
    plt.subplot(5, 1, 2)
    plt.title('Sum of All Gallons Consumed by Street')
    plt.xticks(rotation = 90)
    sns.barplot(data = saws_places_sum, x = saws_places_sum.index, y = 'gallons_consumed', palette = "viridis")
    plt.xlabel('Street Name')
    plt.ylabel('Sum of Monthly Gallons')
    
    plt.subplot(5, 1, 3)
    plt.title('Mean Property Consumption by Month Over 4 Year Period')
    plt.xticks(rotation = 90)
    sns.barplot(data = saws_month_year, x = saws_month_year['Mon_Year'], y = 'gallons_consumed', palette = "viridis")
    plt.xlabel('Month')
    plt.ylabel('Mean Property Consumption')
    
    plt.subplot(5, 1, 4)
    plt.title('Total Consumption by Month Over 4 Year Period in Medical Center')
    plt.xticks(rotation = 90)
    sns.barplot(data = saws_month_year_sum, x = saws_month_year_sum['Mon_Year'], y = 'gallons_consumed', palette = "viridis")
    plt.xlabel('Month')
    plt.ylabel('Total Consumption')
    
    plt.subplot(5, 1, 5)
    plt.title('Mean Property Consumption by Month of Year')
    sns.barplot(data = saws_year_month_mean, x = 'month', y = 'gallons_consumed', palette = "viridis")
    plt.xlabel('Month')
    plt.ylabel('Mean Property Consumption')

In [ ]:
show_saws()

# Sound

In [ ]:
sound_df = wrangle.wrangle_sound()
sound_df = sound_df.set_index('DateTime')
sound_df = sound_df.sort_index()

In [ ]:
sound_df.describe()

In [ ]:
# Plot showing the average daily sound

plt.subplots(figsize=(22, 6))
plt.xticks(rotation = 90)
sns.lineplot(data = sound_df.resample('D').mean(), x = sound_df.resample('D').mean().index, y = 'NoiseLevel_db', palette = "magma")

In [ ]:
plt.plot(sound_df.index, sound_df.NoiseLevel_db)

In [ ]:
hourly_sound = sound_df.resample('H').mean()
hourly_sound

In [ ]:
# Plot of average sound levels by hour of the day

hourly_sound = sound_df.resample('H').mean()
hour_of_day_sound = hourly_sound.groupby(hourly_sound.index.hour).mean()
plt.subplots(figsize=(22, 6))
sns.barplot(data = hour_of_day_sound, x = hour_of_day_sound.index, y = 'NoiseLevel_db', palette = "viridis")

In [ ]:
sound_by_day = sound_df.groupby(sound_df.index.day_name()).mean()
plt.subplots(figsize=(22, 6))
sns.barplot(data = sound_by_day, x = sound_by_day.index, y = 'NoiseLevel_db', palette = "viridis")

# Flood

In [ ]:
flood_df = wrangle.clean_flood()
flood_df = flood_df.set_index('datetime')
flood_df.describe()

In [ ]:
plt.subplots(figsize=(22, 6))
sns.lineplot(data = flood_df.resample('D').mean(), palette = "magma")

# Air

In [ ]:
air_df = wrangle.clean_air()
air_df.datetime = pd.to_datetime(air_df.datetime)
air_df = air_df.set_index('datetime')
air_df = air_df.sort_index()

In [ ]:
air1_df = air_df[(air_df['Pm1_0'] < 500) & (air_df['Pm1_0'] > -50)]
air1_day_df = air1_df.resample('D').mean()
plt.subplots(figsize=(22, 6))
sns.lineplot(data = air1_day_df, x = air1_day_df.index, y = 'Pm1_0', palette = "magma")

In [ ]:

plt.subplots(figsize=(22, 6))
sns.barplot(data = air_by_day, x = air_by_day.index, y = 'Pm1_0', palette = "viridis")

In [ ]:
plt.subplots(figsize=(22, 6))
sns.barplot(data = air_by_day, x = sound_by_day.index, y = 'Pm2_5', palette = "viridis")

In [ ]:
plt.subplots(figsize=(22, 6))
sns.barplot(data = air_by_day, x = sound_by_day.index, y = 'Pm10', palette = "viridis")

In [ ]:
plt.subplots(figsize=(22, 6))
sns.barplot(data = air_by_day, x = sound_by_day.index, y = 'CO', palette = "viridis")

In [ ]:
def show_air():
    air_df = wrangle.clean_air()
    air_df.datetime = pd.to_datetime(air_df.datetime)
    air_df = air_df.set_index('datetime')
    air_df = air_df.sort_index()
    air1_df = air_df[(air_df['Pm1_0'] < 500) & (air_df['Pm1_0'] > -50)]
    air1_day_df = air1_df.resample('D').mean()
    air_df = air_df[(air_df['Pm1_0'] < 500) & (air_df['Pm1_0'] > -50)]
    air_df = air_df[(air_df['Pm2_5'] < 500) & (air_df['Pm2_5'] > -50)]
    air_df = air_df[(air_df['Pm10'] < 500) & (air_df['Pm10'] > -50)]
    air_by_day = air_df.groupby(air_df.index.day_name()).mean()

    plt.subplots(5, 1, figsize=(24, 40), sharey=True)
    plt.subplots_adjust(hspace=.6)
    sns.set(style="darkgrid")
    
    plt.subplot(5, 1, 1)
    plt.title('Daily Mean PM 1.0 Levels')
    sns.lineplot(data = air1_day_df, x = air1_day_df.index, y = 'Pm1_0')
    plt.xlabel('Date')
    plt.ylabel('Reading')
    
    plt.subplot(5, 1, 2)
    plt.title('Mean PM 1.0 Levels by Day of Week')
    sns.barplot(data = air_by_day, x = air_by_day.index, y = 'Pm1_0', palette = "magma")
    plt.xlabel('Day')
    plt.ylabel('Reading')
    
    plt.subplot(5, 1, 3)
    plt.title('Mean PM 2.5 Levels by Day of Week')
    sns.barplot(data = air_by_day, x = air_by_day.index, y = 'Pm2_5', palette = "magma")
    plt.xlabel('Day')
    plt.ylabel('Reading')
    
    plt.subplot(5, 1, 4)
    plt.title('Mean PM 10 Levels by Day of Week')
    sns.barplot(data = air_by_day, x = air_by_day.index, y = 'Pm10', palette = "magma")
    plt.xlabel('Day')
    plt.ylabel('Reading')
    
    plt.subplot(5, 1, 5)
    plt.title('Mean Carbon Monoxide Levels by Day of Week')
    sns.barplot(data = air_by_day, x = air_by_day.index, y = 'CO', palette = "magma")
    plt.xlabel('Day')
    plt.ylabel('Reading')

In [ ]:
show_air()

In [ ]:
pd.read_csv('jan_2018.csv')['b.long'].unique().tolist()

In [ ]:
pd.read_csv('jan_2018.csv')

In [ ]:
def clean_flood(flood):
    '''Drops unneeded columns from the med center flooding df
    Makes sure DateTime is in DateTime format'''
    # drop the columns
    flood = flood.drop(columns=['LAT', 'LONG', 'Zone',  
                          'SensorStatus', 'AlertTriggered', 
                          'Temp_C', 'Temp_F', 'Vendor'])
    # Set to date time format
    flood.DateTime = pd.to_datetime(flood.DateTime)
    flood = flood.rename(columns={"DateTime": "datetime", 
                        "DistToWL_ft": "sensor_to_water_feet", 
                        "DistToWL_m": "sensor_to_water_meters", 
                        "DistToDF_ft": "sensor_to_ground_feet",
                        "DistToDF_m": "sensor_to_ground_meters"})
    # replae -999 with 0
    flood["sensor_to_ground_feet"].replace({-999:13.5006561680}, inplace=True)
    flood["sensor_to_ground_meters"].replace({-999:4.115}, inplace=True)
    
    #flood = flood.replace(to_replace=-999, value=0)
    # create new features for flood depth
    flood['flood_depth_feet'] = flood.sensor_to_ground_feet - flood.sensor_to_water_feet
    flood['flood_depth_meters'] = flood.sensor_to_ground_meters - flood.sensor_to_water_meters 
    # Create new alert
    def flood_alert(c):
        if 0 < c['flood_depth_feet'] < 0.66667:
            return 'No Risk'
        elif 0.66667 < c['flood_depth_feet'] < 1.08333:
            return 'Minor Risk'
        elif 1.08333 < c['flood_depth_feet'] < 2.16667:
            return 'Moderate Risk'
        elif 2.16667 < c['flood_depth_feet']:
            return 'Major Risk !'
        else:
            return 'No Alert'
    flood['flood_alert'] = flood.apply(flood_alert, axis=1)
    flood = flood[(flood.sensor_to_water_feet != -999)]
    # return new df
    return flood

In [ ]:
def clean_air(air):
    '''Drops unneeded columns from the air quality df
    then handles the nulls in alert triggered column
    set to date time format'''
    # drop the colums
    air = air.drop(columns=['LAT', 'LONG', 'Zone', 
                            'Sensor_id', 'SensorModel', 
                            'SensorStatus', 'Vendor'])
    # replace nulls in ALertTriggered to None
    air.fillna("None", inplace = True)
    # set to date time format
    air.DateTime = pd.to_datetime(air.DateTime)
    # rename features
    air = air.rename(columns={"DateTime": "datetime",
                              "AlertTriggered":"alert_triggered"})
    air = air.replace(to_replace=-999, value=0)
    # create time series features
    air['dates'] = pd.to_datetime(air['datetime']).dt.date
    air['time'] = pd.to_datetime(air['datetime']).dt.time
    air['hour'] = pd.to_datetime(air['datetime']).dt.hour
    air['weekday'] = pd.to_datetime(air['datetime']).dt.weekday
    # make all CO bins
    air['AQI_CO'] = pd.cut(air.CO, 
                            bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                            labels = ['Good', 'Moderate', 
                                      'Unhealthy for Sensitive Groups', "Unhealthy", 
                                      "Very Unhealthy", 'Hazardous'])
    
    CO_24hr = air.groupby('dates', as_index=False)['CO'].mean()
    CO_24hr = CO_24hr.rename(columns={'CO':'CO_24hr'})
    air = air.merge(CO_24hr, on = 'dates', how ='left')
    air['AQI_CO_24hr'] = pd.cut(air.CO_24hr, 
                                bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    
    air['AQI_pm2_5'] = pd.cut(air.Pm2_5, 
                                bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    pm_25_24hr = air.groupby('dates', as_index=False)['Pm2_5'].mean()
    pm_25_24hr = pm_25_24hr.rename(columns={'Pm2_5':'Pm_25_24hr'})
    air = air.merge(pm_25_24hr, on = 'dates', how ='left')
    air['AQI_pm_25_24hr'] = pd.cut(air.Pm_25_24hr, 
                                bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    
    air['AQI_pm10'] = pd.cut(air.Pm10, 
                                bins = [-1,55,154,255,355,425,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    pm_10_24hr = air.groupby('dates', as_index=False)['Pm10'].mean()
    pm_10_24hr = pm_10_24hr.rename(columns={'Pm10':'Pm_10_24hr'})
    air = air.merge(pm_10_24hr, on = 'dates', how ='left')
    air['AQI_pm10_24hr'] = pd.cut(air.Pm_10_24hr, 
                                bins = [-1,55,154,255,355,425,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    return air

In [ ]:
def wrangle_weather(weather):
    '''
    This function will drop unneccessary columns, 
    change datetime to a pandas datetime datatype,
    and rename columns to be more readable to return
    a clean dataframe.  
    '''
    #read csv and turn into pandas dataframe
    sa_weather = pd.read_csv('SA_weather.csv')
    # concat sa date and time
    sa_weather['Date_Time'] = sa_weather['Date'] + ' ' + sa_weather['Time']
    # put into date time format
    sa_weather.Date_Time = pd.to_datetime(sa_weather.Date_Time)
    # round to nearest hour
    sa_weather['DateTime'] = sa_weather['Date_Time'].dt.round('60min')
    # set sa weather index
    sa_weather = sa_weather.set_index('DateTime')
    # drop old datetime
    sa_weather = sa_weather.drop(columns=['Date_Time', 'Temp', 'Humidity', 'Barometer'])
    # rename
    sa_weather = sa_weather.rename(columns={"Time": "time", 
                            "Date": "date", 
                            "Weather": "weather", 
                            "Wind": "wind",
                            "Visibility": "visibility"})
    #drop columns we will not be using
    weather.drop(columns=[
    'Sensor_id', 
    'Vendor', 
    'SensorModel', 
    'LAT', 
    'LONG', 
    'Zone', 
    'AlertTriggered', 
    'SensorStatus'], inplace=True)
    #rename columns to be more readable
    weather = weather.rename(columns={"DateTime": "datetime", 
                            "Temp_C": "celsius", 
                            "Temp_F": "farenheit", 
                            "Humidity": "humidity",
                            "DewPoint_C": "dewpoint_celsius",
                            "DewPoint_F": "dewpoint_farenheit",
                            "Pressure_Pa": "pressure"})
    #change datetime to pandas datetime object
    weather.datetime = pd.to_datetime(weather.datetime)
    # round to hour
    weather['DateTime'] = weather['datetime'].dt.round('60min')
    # set index
    weather = weather.set_index('DateTime')
    # join the 2 df's
    weather = weather.join(sa_weather, how='right')
    # repalce -999
    weather = weather.replace(to_replace=-999, value=0)
    # drop nulls
    weather.dropna(inplace = True)
    #return clean weather df
    return weather

In [ ]:
def wrangle_sound(df):
    '''
    This function drops unnecessary columns and
    converts the 'DateTime' column to a datetime 
    object
    '''

    # Drops unnecessary columns
    df = df.drop(columns = ['SensorStatus', 'AlertTriggered', 'Zone', 'LONG', 
                            'LAT', 'SensorModel', 'Vendor', 'Sensor_id'])
    # Converts to datetime
    df['DateTime'] = pd.to_datetime(df.DateTime)
    # make noise level feature
    df['how_loud'] = pd.cut(df.NoiseLevel_db, 
                                bins = [-1,46,66,81,101,4000],
                                labels = ['Normal', 'Moderate', 
                                          'Loud', "Very Loud", 
                                          "Extremely Loud"])
    def sound_alert(c):
        if c['NoiseLevel_db'] > 80:
            return 'Minor Risk'
        elif c['NoiseLevel_db'] > 120:
            return 'Major Risk'
        else:
            return 'No Alert'
    df['sound_alert'] = df.apply(sound_alert, axis=1)
    return df

In [ ]:
def full_daily_downtown_COSA_dataframe():
    
    '''
    This function takes in all COSA dataframes,
    averages them by day, then joins them all together
    using the day as a primary key
    '''

    # Pulls sound CSV and sets datetime as index, then orders it
    df = pd.read_csv('downtown_sound.csv')
    sound_df = wrangle_sound(df)
    sound_df = sound_df.set_index('DateTime')
    sound_df = sound_df.sort_index()
    # Pulls flood CSV and sets datetime as index
    flood = pd.read_csv('downtown_flood.csv')
    flood_df = clean_flood(flood)
    flood_df = flood_df.set_index('datetime')
    # Pulls weather CSV
    weather = pd.read_csv('downtown_weather.csv')
    weather_df = wrangle_weather(weather)
    # Pulls air CSV, sets datetime column to datetime object, sets it as an index, then sorts it
    air = pd.read_csv('downtown_air.csv')
    air_df = clean_air(air)
    air_df.datetime = pd.to_datetime(air_df.datetime)
    air_df = air_df.set_index('datetime')
    air_df = air_df.sort_index()
    # Resamples each dataframe by the day using mean, and drops unnecessary columns from air_df
    weather_day_df = weather_df.resample('D', on='datetime').mean()
    flood_day_df = flood_df.resample('D').mean()
    sound_day_df = sound_df.resample('D').mean()
    air_day_df = air_df.resample('D').mean().drop(columns = ['hour', 'weekday', 'CO_24hr', 'Pm_25_24hr', 'Pm_10_24hr', 'SO2', 'O3', 'NO2'])
    # Creating series for each pollutant
    air2_5 = air_df.drop(air_df.columns.difference(['Pm2_5', 'AQI_pm2_5']), 1)
    air10 = air_df.drop(air_df.columns.difference(['Pm10', 'AQI_pm10']), 1)
    airCO = air_df.drop(air_df.columns.difference(['CO', 'AQI_CO']), 1)
    # Pull most hazardous levels of pollution for each day
    series2_5 = air2_5.resample('D').max().rename(columns = {'AQI_pm2_5': 'most_hazardous_pm2.5_level'})['most_hazardous_pm2.5_level']
    series10 = air10.resample('D').max().rename(columns = {'AQI_pm10': 'most_hazardous_pm10_level'})['most_hazardous_pm10_level']
    seriesCO = airCO.resample('D').max().rename(columns = {'AQI_CO': 'most_hazardous_CO_level'})['most_hazardous_CO_level']
    # Joins the series together in a dataframe
    hazards = pd.DataFrame(series2_5).join(series10).join(seriesCO)
    # Joins the resampled dataframes together
    df = weather_day_df.join(air_day_df).join(hazards).join(sound_day_df).join(flood_day_df)
    # Rounds numbers in specific columns
    df = df.round({'celsius': 2, 'farenheit': 2, 'humidity': 2, 'dewpoint_celsius': 2, 'dewpoint_farenheit': 2,
          'pressure': 2, 'NoiseLevel_db': 2, 'sensor_to_water_feet': 2, 'sensor_to_water_meters': 2,
          'sensor_to_ground_feet': 2, 'sensor_to_ground_meters': 2, 'flood_depth_feet': 2,
          'flood_depth_meters': 2})
    # Create AQI for CO
    df['AQI_CO'] = pd.cut(df.CO, 
                            bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                            labels = ['Good', 'Moderate', 
                                      'Unhealthy for Sensitive Groups', "Unhealthy", 
                                      "Very Unhealthy", 'Hazardous'])
    # create AQi for pm 2.5
    df['AQI_pm2_5'] = pd.cut(df.Pm2_5, 
                                bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    # create AQI for pm 10
    df['AQI_pm10'] = pd.cut(df.Pm10, 
                                bins = [-1,55,154,255,355,425,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    return df



In [ ]:
df = full_daily_downtown_COSA_dataframe()

In [ ]:
df

In [ ]:
wrangle_COSA.full_daily_downtown_COSA_dataframe()

In [ ]:
wrangle_COSA.full_daily_brooks_COSA_dataframe()

In [ ]:
wrangle_COSA.full_daily_medcenter_COSA_dataframe()

In [ ]:
# Pulls the data
saws_df = wrangle.wrangle_saws()
# Creates a new column that has the date in a "year month" format that can be converted to a datetime object
saws_df['year_month'] = '20' + saws_df['year_month']
# Converts to datetime
saws_df['year_month'] = pd.to_datetime(saws_df['year_month'])
# Sets the index to the datetime object, then groups the data by month and drops zipcode (since they are all in the same one)
saws_month_year = saws_df.set_index('year_month').resample('M').mean().drop(columns = ['zipcode'])
# Creates a new column for labeling graphs with month and year
saws_month_year['Date'] = pd.to_datetime(saws_month_year.index)
# Truncates the 'Date' column into just month and year
saws_month_year['Mon_Year'] = saws_month_year['Date'].dt.strftime('%b-%Y')
# Creates a new dataframe for the mean of gallons used by street, drops zipcode column
saws_places = saws_df.groupby('location').mean().drop(columns =['zipcode']).reset_index()
# Creates a new dataframe for the sum of gallons used by street
#saws_places_sum = saws_df.groupby('location').sum()
#import calendar
#saws_df = saws_df.replace('*', 0)
#saws_df['gallons_consumed'] = pd.to_numeric(saws_df['gallons_consumed'])
## Creates dataframe for repesenting months by mean monthly water usage
#saws_year_month_mean = saws_df.gallons_consumed.groupby(saws_df['year_month'].dt.month).mean()
#saws_year_month_mean = pd.DataFrame(saws_year_month_mean)
#saws_year_month_mean['month'] = saws_year_month_mean.index
## Labels numeric months into their respective shortened titles
#saws_year_month_mean['month'] = saws_year_month_mean['month'].apply(lambda x: calendar.month_abbr[x])
#plt.subplots(4, 1, figsize=(24, 40), sharey=True)
#plt.subplots_adjust(hspace=.5)
#sns.set(style="darkgrid")
#    
#plt.subplot(4, 1, 1)
#plt.title('Average Monthly Water Use by Street')
#plt.xticks(rotation = 90)
#sns.barplot(data = saws_places, x = 'location', y = 'gallons_consumed', palette = "viridis")
#plt.xlabel('Street Name')
#plt.ylabel('Average Monthly Gallons')

In [ ]:
saws_places

In [ ]:
saws_df.groupby('location').mean()

In [ ]:
wrangle.show_saws()